In [1]:
import numpy as np
#based on gameboard and car letter return x or y
def moveAxis(gameboard,car): 
    for char in range (len(gameboard)):
        if gameboard[char]==car and p1==null:  
            p1=char
        if gameboard[char]==car and p1!=null:
            p2=char
    if (p1-p2)==-6:
        return y
    if (p1-p2)==-1:
        return x

    
#read text file and return array 1d list of gameboard and dict of fuel level 
def readfile(filename):
    gameboards=[]
    carFuels=[]
    f = open(filename, "rt")
    count=0
    for x in f:
        gameboard=[]
        carFuel={}
        if x[0]=="#":
            continue
        if len(x)==1:
            continue
        else:
            x=x.strip("\n")
            token=x.split()
            
            for i in range(len(token)):        
                if i==0:
                    count+=1
                    for j in range(36):
                        gameboard.insert(len(gameboard),token[i][j])
                        if token[i][j] not in  carFuel and  token[i][j] !='.':
                            carFuel[token[i][j]]=100
                elif i>=1: 
                    
                    car=token[i][0]
                    fuelLevel=token[i][1]
                    carFuel.update({car:fuelLevel})
        gameboard=np.array(gameboard).reshape(6,6).tolist()
        gameboards.insert(count-1,gameboard)
        carFuels.insert(count-1,carFuel)        
                
                
           
    f.close()
    return gameboards,carFuels
#adds the valet service to the game
def valetService(gameboard,car):
    if gameboard[17]!=".":
        if(gameboard[17]==car and gameboard[16]==car):
            gameboard[:] = [x if x != car else "." for x in gameboard]
    return gameboard    
    
    
    


In [8]:

gameboards,fuel=readfile("sample-input.txt")
for i in range(len(gameboards)):
    print("initial board configuration for board " , i+1)
    for j in range(len(gameboards[i])):
        print(gameboards[i][j])
    print("Car fuel available", fuel[i])
    print()

initial board configuration for board  1
['B', 'B', 'I', 'J', '.', '.']
['.', '.', 'I', 'J', 'C', 'C']
['.', '.', 'I', 'A', 'A', 'M']
['G', 'D', 'D', 'K', '.', 'M']
['G', 'H', '.', 'K', 'L', '.']
['G', 'H', 'F', 'F', 'L', '.']
Car fuel available {'B': 100, 'I': 100, 'J': 100, 'C': 100, 'A': 100, 'M': 100, 'G': 100, 'D': 100, 'K': 100, 'H': 100, 'L': 100, 'F': 100}

initial board configuration for board  2
['.', '.', 'I', '.', '.', '.']
['B', 'B', 'I', '.', 'K', '.']
['G', 'H', 'A', 'A', 'K', 'L']
['G', 'H', 'D', 'D', 'K', 'L']
['G', '.', '.', 'J', 'E', 'E']
['F', 'F', '.', 'J', '.', '.']
Car fuel available {'I': 100, 'B': 100, 'K': 100, 'G': 100, 'H': 100, 'A': 100, 'L': 100, 'D': 100, 'J': 100, 'E': 100, 'F': 100}

initial board configuration for board  3
['J', 'B', 'B', 'C', 'C', 'C']
['J', 'D', 'D', '.', '.', 'M']
['J', 'A', 'A', 'L', '.', 'M']
['F', 'F', 'K', 'L', '.', 'N']
['.', '.', 'K', 'G', 'G', 'N']
['.', 'H', 'H', '.', '.', '.']
Car fuel available {'J': 100, 'B': 100, 'C': 10

In [ ]:
#def Uniform Cost Search (UCS):

    
#def Greedy Best First Search (GBFS):

    
#def Algorithm A or A⋆(A/A⋆):